# Hugging Face Pipeline 개요 및 활용

## 1. Pipeline API 개념 및 활용 범위  
Hugging Face의 `pipeline` API는 NLP 모델 활용을 간소화하고 추상화한 고수준 인터페이스입니다.  
- **개념**: 복잡한 모델 로딩, 토큰화, 추론 과정을 한 번에 처리하는 간단한 함수 호출 형태의 API.  
- **장점**: 모델 활용 시 필수적인 전처리(토큰화) → 모델 추론 → 후처리(결과 해석) 과정을 모두 내부적으로 처리하므로, 사용자는 모델 사용 로직을 간단히 유지할 수 있습니다.  
- **활용 범위**: 감정 분석(Sentiment Analysis), 제로샷 분류(Zero-Shot Classification), 질의응답(Question Answering), 번역(Translation), 요약(Summarization), 키워드 추출 등 다양한 NLP 태스크에 쉽게 적용할 수 있습니다.

## 2. 다양한 Pipeline 사례  
- **Sentiment Analysis**: 입력 문장의 감정(긍정, 부정 등)을 추론하는 파이프라인.  
- **Zero-Shot Classification**: 미리 정의되지 않은 레이블에 대해 문장을 분류할 수 있는 파이프라인.  
- **Question-Answering(QA)**: 문서(컨텍스트)와 질문을 바탕으로 해당하는 답을 추론하는 파이프라인.  
- **Summarization**: 긴 문서를 요약문으로 압축하는 파이프라인.  
- **Translation**: 하나의 언어로 된 문장을 다른 언어로 번역하는 파이프라인.

여기서는 특히 **질의응답(Question Answering) Pipeline**에 초점을 맞춰 살펴보겠습니다.

## 3. 질의응답(QA) Pipeline 개요  
**Question-Answering Pipeline**은 주어진 문서(컨텍스트)와 하나의 질문을 입력받고, 해당 문서 내에서 답을 추출하는 파이프라인입니다.  
- 입력:  
  - `context`: 질문에 대한 해답을 포함하고 있을 것으로 기대되는 텍스트 문서  
  - `question`: 답을 얻고자 하는 특정 질문  
- 출력:  
  - 문서 내에서 추출한 정답 문자열과 해당 정답의 점수, 위치 정보 등을 제공합니다.

## 4. QA Pipeline 사용법


In [1]:
from transformers import pipeline
# 'question-answering' 파이프라인을 로드합니다. 여기서 모델과 토크나이저는
# 사전에 학습된 QA 모델(기본값: 'distilbert-base-cased-distilled-squad')을 기본 사용.
# 인자로 model, tokenizer, device 등을 직접 설정 가능
qa_pipeline = pipeline('question-answering')

# 파이프라인 객체에서 모델과 토크나이저 접근하기
model = qa_pipeline.model
tokenizer = qa_pipeline.tokenizer

print("사용 중인 모델:", model.__class__.__name__)
print("모델 이름 혹은 경로:", model.name_or_path)
print("사용 중인 토크나이저:", tokenizer.__class__.__name__)

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


사용 중인 모델: DistilBertForQuestionAnswering
모델 이름 혹은 경로: distilbert/distilbert-base-cased-distilled-squad
사용 중인 토크나이저: DistilBertTokenizerFast


In [2]:
result = qa_pipeline({
    'context': "Hugging Face는 NLP 분야에서 매우 유명한 오픈소스 커뮤니티이자 기업이다.",
    'question': "Hugging Face는 어떤 분야에서 유명한가?"
})
print(result)
# 결과 예시:
# {'score(신뢰도)': 0.95, 'start(시작 인덱스)': 18, 'end(끝 인덱스)': 21, 'answer(정답 텍스트)': 'NLP'}
# 위 예제에서, context와 question을 dict 형태로 전달하면 파이프라인이 내부적으로
# 토큰화 → 모델추론 → 답변 추출을 진행한 뒤 answer를 반환합니다.

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


{'score': 0.8067139387130737, 'start': 14, 'end': 17, 'answer': 'NLP'}


### 파이프라인 생성 시 원하는 모델 지정하기

In [3]:
# 특정 모델 이름 지정(예: 'bert-base-uncased'를 QA 모델로 활용)
# 단, "question-answering" 태스크는 SQuAD 등 QA용으로 파인튜닝된 모델을 쓰는 것이 일반적이므로
# 기본 BERT 모델을 바로 쓰면 성능이 낮을 수 있음.
# 여기서는 QA 태스크용으로 학습된 모델 예를 들어봄: 'deepset/roberta-base-squad2'
qa_pipeline_custom = pipeline(
    "question-answering",
    model="deepset/roberta-base-squad2",
    tokenizer="deepset/roberta-base-squad2"
)

print("사용 중인 모델:", qa_pipeline_custom.model.__class__.__name__)
print("모델 이름 혹은 경로:", qa_pipeline_custom.model.name_or_path)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


사용 중인 모델: RobertaForQuestionAnswering
모델 이름 혹은 경로: deepset/roberta-base-squad2


## 5. 파이프라인 동작 흐름 이해
1. **문서 입력**: 사용자가 질의응답 대상이 되는 텍스트(컨텍스트)와 질문을 입력.
2. **토큰화**: context와 question을 합쳐 모델이 이해할 수 있는 토큰 형태로 변환.
3. **모델 추론**: 사전 학습된 QA 모델(BERT 기반 등)을 통해 question과 context를 함께 입력받아 answer span을 예측.
4. **답변 추출**: 예측된 시작 토큰과 종료 토큰 위치를 기반으로 context 문서에서 해당 범위의 텍스트를 추출하여 최종 답변으로 제공.

## 6. 간단한 질의응답 봇 파인튜닝 실습
짧은 위키피디아 문서를 하나 준비합니다.
예: 위키피디아의 "Python(언어)" 문서 일부 발췌.
사용자로부터 질문을 입력받고, QA pipeline에 해당 질문과 문서를 넣어 모델의 답변을 확인합니다.
문서를 바꾸거나 질문을 바꿔가며 모델 반응을 살펴보고, 정확하지 않은 답변 시나리오를 통해 모델의 한계점을 파악해봅니다.

In [4]:
# Hugging Face에서 제공하는 Transformers 라이브러리를 임포트합니다.
# Transformers: 사전 학습된 NLP 모델과 토크나이저 등 다양한 기능을 제공하는 라이브러리
from transformers import pipeline

# question-answering 파이프라인을 불러옵니다.
# pipeline('question-answering')은 기본적으로 QA를 수행하기 위한 모델과 토크나이저를 자동으로 로드합니다.
qa_pipeline = pipeline('question-answering')

# 아래 context는 위키피디아의 Python 언어 항목 중 일부를 발췌한 예시입니다.
# (출처: Wikipedia "Python (programming language)" 문서 일부, 가상의 예시)
context = """
Python은 1991년 귀도 반 로섬(Guido van Rossum)이 발표한 인터프리터 방식의 프로그래밍 언어로,
코드 가독성이 우수하고 다양한 프로그래밍 패러다임을 지원한다.
"""

# 함수 예제: 질의응답을 수행하는 함수
# 입력: (context: str, question: str)
# 출력: dict 형태로 {'answer': str, 'score': float, 'start': int, 'end': int} 형식의 답변 정보
def answer_question(context, question):
    # 파이프라인에 context와 question을 dict 형태로 전달
    result = qa_pipeline({
        'context': context,
        'question': question
    })
    return result

# 예시 질의응답 수행
# question: "Python은 누가 발표했는가?"
# 기대되는 답변: "귀도 반 로섬(Guido van Rossum)"
question = "Python은 누가 발표했는가?"
result = answer_question(context, question)

# 결과 출력
print("질문:", question)
print("답변:", result['answer'])
print("점수(정확도 추정치):", result['score'])

# 다른 질문 시도
# question: "Python은 언제 발표되었는가?"
# 기대 답변: "1991년"
question = "Python은 언제 발표되었는가?"
result = answer_question(context, question)
print("질문:", question)
print("답변:", result['answer'], "(정확도:", result['score'], ")")

# 모델 한계점 관찰을 위한 엉뚱한 질문
# context에 없는 정보 질문: "Python은 어느 행성에서 유래했는가?"
# 이 경우 문맥에 없는 정보이므로 답변이 어색하거나 문맥과 무관하게 추출될 수 있음.
question = "Python은 어느 행성에서 유래했는가?"
result = answer_question(context, question)
print("질문:", question)
print("답변:", result['answer'], "(정확도:", result['score'], ")")

# 이처럼 없는 정보에 대한 질문이나 모호한 질문, 사전학습 데이터와 관련이 적언 언어에 대해 모델이 부정확한 답변을 내놓을 수 있습니다.

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


질문: Python은 누가 발표했는가?
답변: Guido van Rossum
점수(정확도 추정치): 0.41587844491004944
질문: Python은 언제 발표되었는가?
답변: Guido van Rossum (정확도: 0.41587844491004944 )
질문: Python은 어느 행성에서 유래했는가?
답변: Guido van Rossum (정확도: 0.3983064591884613 )


## 7. 모델 한계점 및 에러 케이스
문서 안에 없는 정보에 대한 질문을 하면, 엉뚱한 답변을 하거나 최적의 답변을 찾지 못할 수 있습니다.
문장이 애매하거나 중의적이면 모델이 문맥 파악을 제대로 하지 못할 수 있습니다.
전문용어, 코드 예시, 숫자 등 특정 타입의 정보에 대해 정확도가 떨어질 수 있습니다.


### 간단한 데이터로 파인튜닝(Fine-tuning)하기

- Hugging Face Transformers는 파인튜닝을 위해 Trainer API를 제공합니다.
- 다음은 SQuAD 같은 질의응답 데이터셋 일부를 이용해 파인튜닝하는 간단한 예시 흐름입니다.

**파인튜닝 개념**
- 이미 QA 태스크로 사전학습(fine-tuned)된 모델을 더 작은 새로운 데이터에 맞춰 추가 훈련하는 과정.
- 예시: 특정 도메인(의료, 법률)에 특화된 데이터로 파인튜닝하여 해당 도메인 질문에 더 정확히 답하도록 함.

**실습**: QA 모델 파인튜닝(요약된 흐름)
데이터 준비
- context, question, answers(정답 범위를 지정한 형태)로 구성된 JSON 또는 CSV 형태 데이터 준비.
- SQuAD 포맷 예시:
```{
  "data": [
    {
      "title": "Example Title",
      "paragraphs": [
        {
          "context": "여기는 파인튜닝용 문서 컨텍스트입니다.",
          "qas": [
            {
              "id": "1",
              "question": "여기는 무엇을 위한 문서인가?",
              "answers": [
                {
                  "text": "파인튜닝용 문서",
                  "answer_start": 4
                }
              ]
            }
          ]
        }
      ]
    }
  ]
}
```

아래 코드는 설명을 위한 예시이며, 실제로는 다양한 설정(하이퍼파라미터, 데이터셋 경로, 모델 경로) 조정이 필요합니다.

In [ ]:
!pip install datasets

In [4]:
# datasets: 다양한 NLP 데이터셋 로드 및 처리 라이브러리
# transformers: Hugging Face Transformers로 모델, 토크나이저, Trainer 등 사용
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer

# 1) 데이터 로드: KorQuAD (https://huggingface.co/datasets/KorQuAD/squad_kor_v1)
# KorQuAD는 한국어 기계독해(MRC) 데이터셋으로, SQuAD 형식을 따릅니다.
# 전체 데이터를 로드하면 용량이 크기 때문에 여기서는 'validation' split만 로드합니다.
val_data = load_dataset('KorQuAD/squad_kor_v1', split='validation')

# 2) 데이터 분할: validation 데이터를 8:2 비율로 train과 test로 나눕니다.
# train_test_split 함수는 Dataset 객체를 나누어 DatasetDict 형태로 반환합니다.
split_data = val_data.train_test_split(test_size=0.2)

# 분할된 데이터셋을 각각 변수에 저장:
train_dataset = split_data['train']  # 전체 데이터의 80%를 훈련용으로 사용.
valid_dataset = split_data['test']   # 나머지 20%를 검증용으로 사용.

# 3) KoELECTRA 기반 KorQuAD 모델의 토크나이저 로드
# monologg/koelectra-base-v3-finetuned-korquad: KoELECTRA 모델을 KorQuAD 데이터셋으로 미세 조정한 모델입니다.
# AutoTokenizer는 해당 모델의 토크나이저를 자동으로 불러옵니다.
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-finetuned-korquad")

# 4) 전처리 함수 정의: SQuAD 형식에 맞는 데이터 전처리
# 이 함수는 각 데이터셋 예제의 질문(question), 문맥(context), 정답(answers)을 토크나이징하고,
# 모델이 학습할 수 있도록 start_positions와 end_positions를 계산합니다.

def preprocess_function(data):
    """
    데이터셋을 모델 입력 형태로 변환하는 함수입니다.
    질문과 문맥을 토크나이즈하고 정답의 시작/끝 위치를 계산하여 추가합니다.
    """
    tokenized = tokenizer(
        data["question"],        # 질문 텍스트
        data["context"],         # 문맥 텍스트
        truncation=True,         # 최대 길이를 초과하면 문맥을 잘라냄
        padding='max_length',    # 최대 길이에 맞게 패딩 추가
        max_length=384,          # 최대 토큰 길이 (일반적으로 384 사용)
        return_offsets_mapping=True  # 각 토큰의 원본 텍스트 내 시작/끝 위치를 매핑
    )

    # 4-2. 정답의 시작(start_positions)과 끝(end_positions) 인덱스를 저장할 리스트 생성
    start_positions = []
    end_positions = []

    # 4-3. 각 예제에 대해 토큰화된 오프셋과 원본 정답 위치를 비교하여 토큰 인덱스 계산
    for i, offsets in enumerate(tokenized["offset_mapping"]):
        # KorQuAD v1 데이터의 answers 구조:
        # {"text": [정답 문자열], "answer_start": [정답의 시작 위치]}
        answer_start = data["answers"][i]["answer_start"][0]  # 정답 시작 위치 (문서 내 위치)
        answer_text = data["answers"][i]["text"][0]           # 정답 문자열
        answer_end = answer_start + len(answer_text)          # 정답 끝 위치

        # 각 토큰이 문맥에 해당하는지를 나타내는 sequence_ids 구하기:
        # sequence_ids는 각 토큰이 question(0)인지 context(1)인지 구분합니다.
        sequence_ids = tokenized.sequence_ids(i)  # 각 토큰의 sequence id (question/context 구분)
        context_start = sequence_ids.index(1)  # 문맥(context)이 시작하는 첫 토큰의 인덱스
        context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)  # 문맥이 끝나는 마지막 토큰의 인덱스

        # 정답 시작/끝 위치를 포함하는 토큰의 인덱스 초기화
        start_token_index = context_start
        end_token_index = context_start

        # 오프셋을 순회하며 정답 시작과 끝이 포함된 토큰 인덱스를 찾습니다.
        for idx, (offset_start, offset_end) in enumerate(offsets):
            # 정답 시작 위치가 현재 토큰의 범위에 포함되면 start_token_index 설정
            if offset_start <= answer_start < offset_end:
                start_token_index = idx
            # 정답 끝 위치가 현재 토큰의 범위에 포함되면 end_token_index 설정
            if offset_start < answer_end <= offset_end:
                end_token_index = idx
        # 정답의 시작과 끝 토큰 인덱스를 리스트에 추가
        start_positions.append(start_token_index)
        end_positions.append(end_token_index)

    # 4-4. 모델 입력에 필요하지 않은 "offset_mapping" 키를 제거합니다.
    tokenized.pop("offset_mapping", None)
    # 4-5. 모델 학습을 위해 start_positions와 end_positions를 추가합니다.
    tokenized["start_positions"] = start_positions
    tokenized["end_positions"] = end_positions

    # 최종적으로 토크나이즈된 데이터를 반환합니다.
    return tokenized

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.29k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/11.6M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/591 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [7]:
# map 함수를 통해 실제 토큰화 적용
train_tokenized = train_dataset.map(preprocess_function, batched=True)
valid_tokenized = valid_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/4619 [00:00<?, ? examples/s]

Map:   0%|          | 0/1155 [00:00<?, ? examples/s]

In [8]:
# 결과 확인
print(valid_tokenized[0]['context'])  # 원본 텍스트
print(valid_tokenized[0]['question'])
print(valid_tokenized[0]['answers'])

5월 27일, 지난 4월 29일 다라 지역에서 반정부 시위에 참여했던 13살 소년 함자 알하티브 (Hamza Al-Khatib) 가 고문 흔적이 가득한 시체로 집에 돌아오자, 시리아인들과 국제 사회의 분노가 빗발쳤다. 5월 29일, 홈스 주의 탈비셰흐와 라스탄에서 바샤르 알아사드 대통령에 반대하는 시위가 벌어졌으며, 정부의 탱크와 중기관총을 동원한 유혈 진압으로 11명이 살해되고, 무수한 사람들이 부상을 입었다. 탈비셰흐와 라스탄 이외에, 다라, 바니야스, 탈칼라크에서도 반정부 시위에 대한 유혈 진압이 계속되었다. 5월 30일, 반정부 시위자들과 정부군 간의 첫 무장 충돌이 일어났으며 , 시리아의 집권당 바트당은 전국민대화위원회를 구성하였다.
반정부 시위자들과 정부군 간의 첫 무장 충돌이 일어난 날짜는 언제인가?
{'text': ['5월 30일'], 'answer_start': [291]}


In [9]:
# 모델 로드
model = AutoModelForQuestionAnswering.from_pretrained("monologg/koelectra-base-v3-finetuned-korquad")

# 트레이닝 설정
training_args = TrainingArguments(
    output_dir="./qa_finetuned_model",
    evaluation_strategy="steps",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    save_steps=10_000,
    save_total_limit=2
)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,  # 실제 key는 데이터 구조에 따라 'train'일 수도 있고 아니면 그냥 [0]에 접근해야 할 수도 있음.
    eval_dataset=valid_tokenized,   # 마찬가지로 데이터 구조 확인 필요
    tokenizer=AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-finetuned-korquad")
)

# 파인튜닝 실행
trainer.train()

model.safetensors:   0%|          | 0.00/449M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-41efcc9e9c78>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss


TrainOutput(global_step=290, training_loss=0.41807542998215247, metrics={'train_runtime': 793.2282, 'train_samples_per_second': 11.646, 'train_steps_per_second': 0.366, 'total_flos': 1810394579045376.0, 'train_loss': 0.41807542998215247, 'epoch': 2.0})

파인튜닝 후 확인
- 파인튜닝 종료 후 trainer.save_model() 호출로 모델 저장 가능.
- 저장된 경로를 이용해 pipeline("question-answering", model="./qa_finetuned_model") 형태로 파이프라인 생성 가능.

파인튜닝된 모델 로드 및 파이프라인 재생성

In [14]:
from transformers import pipeline

# 파인튜닝 완료 후 저장한 모델 경로 사용
finetuned_qa_pipeline = pipeline("question-answering", model="/content/qa_finetuned_model/checkpoint-290")

result = finetuned_qa_pipeline({
    'context': """Transformer 모델은 그 혁신적인 구조와 뛰어난 성능으로 현대 NLP의 중심이 되었습니다.
     병렬 처리 능력과 장거리 의존성 포착 능력은 이전 모델들의 한계를 극복했으며,
     이는 더 큰 모델과 더 다양한 응용으로 이어지고 있습니다.
     앞으로도 Transformer는 AI 발전의 핵심 요소로 계속 진화할 것으로 예상됩니다.""",
    'question': "Transformer는 이전 모델의 어떤 한계를 극복했는가?"
})

print(result['answer'])
# 파인튜닝된 데이터셋에 맞춰 개선된 결과를 기대할 수 있습니다.

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


병렬 처리 능력과 장거리 의존성 포착 능력은


### 정리

- 파이프라인 모델/토크나이저 확인: pipeline 객체의 .model, .tokenizer 속성을 통해 현재 사용 중인 모델 정보 확인 가능.
- 원하는 모델 지정: pipeline 생성 시 model="모델이름", tokenizer="토크나이저이름" 파라미터로 임의의 모델/토크나이저 지정 가능.
- 파인튜닝: Trainer API와 데이터셋을 활용하여 사전 학습된 모델을 특정 데이터셋에 맞춰 재학습할 수 있으며, 완료 후 해당 모델을 파이프라인으로 다시 로드하여 사용 가능.

이로써, 파이프라인의 기본 활용부터 모델 지정, 파인튜닝까지 일련의 과정을 모두 살펴볼 수 있습니다.